In [1]:
import pandas as pd
import json
from geopy.geocoders import Nominatim

rentdata_df = pd.read_json('rent_data.json')

In [2]:
title = []                  # 1
price = []                  # 2
area = []                   # 3
bedroom = []                # 4
description = []            # 5
width = []                  # 6
direction = []              # 7
toilet = []                 # 8
address = []                # 9
geo_code = []               # 10
law = []
floor = []
district = []
province_city = []
type = []
type_estate = []
image_links = []
phonenumber = []

In [ ]:
import requests
import pandas as pd
import re

rentdata_df["geo_code"] = [None for _ in range(len(rentdata_df))]
rentdata_df["width"] = [None for _ in range(len(rentdata_df))]
rentdata_df["direction"] = [None for _ in range(len(rentdata_df))]
rentdata_df["district"] = [None for _ in range(len(rentdata_df))]
rentdata_df["province_city"] = [None for _ in range(len(rentdata_df))]

def get_geocode(address):
    api_key = 'b48b7a0d92434d7ba9e13d0feb2db0ad'
    url = f"https://api.opencagedata.com/geocode/v1/json?q={address}&key={api_key}"

    response = requests.get(url)
    data = response.json()

    if data['results']:
        latitude = data['results'][0]['geometry']['lat']
        longitude = data['results'][0]['geometry']['lng']
        print(f"Latitude: {latitude}, Longitude: {longitude}")
        return [latitude, longitude]
    else:
        # Check if address contains "Đường"
        if "Đường" in address:
            # Remove "Đường" and retry
            modified_address = address.replace("Đường", "").strip()
            print(f"Retry with modified address: {modified_address}")
            return get_geocode(modified_address)
        else:
            print("No results found.")
            return None

def data_normalization(df):
    for i, address in enumerate(df["address"]):
        df.at[i, "geo_code"] = get_geocode(address)
        district, province_city = extract_district_and_province(address)
        df.at[i, "district"] = district.replace("Q.", '').replace("H.", '').replace("Tp.", '').replace("Tx.", '').strip()    
        df.at[i, "province_city"] = province_city.replace("Tp.", '').replace("T.", '').strip()     
        
    for i, price in enumerate(df['price']):
        price = price.strip()
        price = price.replace(',', '.')        
        # Kiểm tra và xử lý nếu tồn tại từ 'Triệu'
        if 'Triệu' in price:
            # Loại bỏ từ 'Triệu' và khoảng trắng
            price = price.replace('Triệu/Tháng', '').strip()      
        df.at[i, 'price'] = price
        
    for i, width in enumerate(df['width']):
        if isinstance(width, str):
            df.at[i, "width"] = width.replace("m", '').strip()
        
    for i, area in enumerate(df['area']):
        if isinstance(area, str):
            df.at[i, "area"] = area.replace("m", '').replace("²", '').strip()
            
    for i, direction in enumerate(df['direction']):
        if isinstance(direction, str):
            df.at[i, "direction"] = direction.split(',')[0].strip() 
    
    for i, phonenumber in enumerate(df['phonenumber']):        
        if isinstance(phonenumber, str):
            phonenumber = phonenumber.replace(' ', '').replace('.', '').strip()
            # Kiểm tra nếu số đầu tiên khác 0 thì thêm số 0 vào đầu
            if phonenumber[0] != '0':
                phonenumber = '0' + phonenumber
            df.at[i, 'phonenumber'] = phonenumber.strip()
            
    for i, title in enumerate(df['title']):
        if isinstance(title, str):
            title = re.sub(r'[^\w\s,.()\–]', '', title)
            title = title.lower()
            title = title.capitalize()
            df.at[i, 'title'] = title.strip()   

def extract_district_and_province(address):
    parts = address.split(',')
    if len(parts) >= 2:
        district = parts[-2].strip()
        province_city = parts[-1].strip()
        return district, province_city
    return None, None

data_normalization(rentdata_df)

rentdata_df['type_estate'] = rentdata_df['type_estate'].replace({
    'Nhà mặt phố': 'Nhà Mặt Phố',
    'Biệt thự': 'Nhà Biệt Thự, Liền Kề',
    'Liền kề': 'Nhà Biệt Thự, Liền Kề',
    'Chung cư mini': 'Căn Hộ Chung Cư',
    'Căn hộ chung cư': 'Căn Hộ Chung Cư',
    'Nhà tập thể': 'Căn Hộ Chung Cư',
    'Đất': 'Nhà Đất Thổ Cư',
    'Nhà riêng': 'Nhà Đất Thổ Cư',
    'Đất nền dự án': 'Đất Nền Dự Án',
    'Dự án BĐS': 'Đất Nền Dự Án',
    'Nhà trọ': 'Nhà Trọ, Phòng Trọ',
    'Phòng trọ': 'Nhà Trọ, Phòng Trọ',
    'Khách sạn - Nhà nghỉ': 'Nhà Hàng, Khách Sạn, Resort',
    'Căn hộ dịch vụ - Homestay': 'Nhà Hàng, Khách Sạn, Resort',
    'Khu nghỉ dưỡng - Resort': 'Nhà Hàng, Khách Sạn, Resort',
    'Căn hộ khách sạn - Condotel': 'Condotel',
    'Căn hộ văn phòng - Officetel': 'OficeTel',    
    'Nhà phố thương mại - Shophouse': 'ShopHouse',
    'Tòa nhà văn phòng': 'Văn phòng, TTTM, Cửa hàng, Kiot',
    'Văn phòng Co-working': 'Văn phòng, TTTM, Cửa hàng, Kiot',
    'Trang trại': 'Đất Trang Trại',
    'Văn phòng': 'Văn phòng, TTTM, Cửa hàng, Kiot',
    'Nhà hàng - Cửa hàng - Kiot': 'Văn phòng, TTTM, Cửa hàng, Kiot',
    'Mặt bằng': 'Văn phòng, TTTM, Cửa hàng, Kiot',
    'Xưởng sản xuất thực phẩm': 'Nhà Xưởng, Mặt Bằng, Kho Bãi',
    'Kho bãi - Nhà xưởng - Khu công nghiệp': 'Nhà Xưởng, Mặt Bằng, Kho Bãi',
    'Mặt bằng': 'Nhà Xưởng, Mặt Bằng, Kho Bãi',
    'Đất trống': 'Nhà Xưởng, Mặt Bằng, Kho Bãi',
})

def write_data(df, field, data_list):
    if not isinstance(data_list, list):
        data_list = []  # Initialize as empty list if not already
    
    if field in df.columns:
        for value in df[field]:
            data_list.append(value)
    else:
        print(f"Error: '{field}' column not found in the DataFrame.")


# Populate lists with data from DataFrame
write_data(rentdata_df, "title", title)
write_data(rentdata_df, "price", price)
write_data(rentdata_df, "area", area)
write_data(rentdata_df, "type", type)
write_data(rentdata_df, "type_estate", type_estate)
write_data(rentdata_df, "image_links", image_links)
write_data(rentdata_df, "bedroom", bedroom)
write_data(rentdata_df, "description", description)
write_data(rentdata_df, "width", width)
write_data(rentdata_df, "direction", direction)
write_data(rentdata_df, "toilet", toilet)
write_data(rentdata_df, "address", address)
write_data(rentdata_df, "law", law)
write_data(rentdata_df, "floor", floor)
write_data(rentdata_df, "geo_code", geo_code)
write_data(rentdata_df, "district", district)
write_data(rentdata_df, "province_city", province_city)
write_data(rentdata_df, "phonenumber", phonenumber)

# Display the updated DataFrame
print("\nUpdated DataFrame:")


In [ ]:
rentdata_df

In [ ]:
rentdata_df.to_csv('rent_data.csv', encoding='utf-8', index=False)

In [ ]:
import pymysql

# Thiết lập kết nối tới cơ sở dữ liệu MySQL
connection = pymysql.connect(
    host='localhost',
    user='root',
    password='123456',
    database='estate',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

try:
    with connection.cursor() as cursor:
        # Xóa dữ liệu từ cả hai bảng pools_bds và pools_image
        cursor.execute("""
            DELETE pb, pi
            FROM pools_bds pb
            JOIN pools_image pi ON pb.BDS_id = pi.BDS_id
            WHERE pb.kind = 'Cho Thuê' AND pi.content LIKE '%.meey%'
        """)
        connection.commit()

finally:
    connection.close()


In [ ]:
import csv
import pymysql

# Thiết lập kết nối tới cơ sở dữ liệu MySQL với mã hóa utf8mb4
connection = pymysql.connect(
    host='localhost',
    user='root',
    password='123456',
    database='estate',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

def is_decimal(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

try:
    with connection.cursor() as cursor:
        
        # Đọc dữ liệu từ file CSV và nhập vào MySQL
        with open('rent_data.csv', 'r', encoding='utf-8') as file:
            csv_data = csv.DictReader(file)
            for row in csv_data:
                try:
                    district = row['district']
                    province_city = row['province_city']
                    
                    # Kiểm tra tồn tại address
                    cursor.execute("SELECT address_id FROM pools_address WHERE district = %s AND province_city = %s", (district, province_city))
                    existing_address = cursor.fetchone()
                    if existing_address:
                        address_id = existing_address['address_id']
                    else:
                        # Nếu không tìm thấy địa chỉ, bỏ qua dòng này
                        print(f"Không tìm thấy địa chỉ cho dòng: {row}")
                        continue
                    
                    # Kiểm tra pools_typeofbds 
                    bds_type = row['type_estate']
                    cursor.execute("SELECT Type_Id FROM pools_typeofbds WHERE TypeName = %s", (bds_type,))
                    existing_type = cursor.fetchone()
                    if existing_type:
                        type_id = existing_type['Type_Id']
                    else:
                        # Nếu không tìm thấy loại bất động sản, bỏ qua dòng này
                        print(f"Không tìm thấy loại bất động sản cho dòng: {row}")
                        continue
                    
                    # Thêm dữ liệu vào bảng pools_bds
                    title = row['title']
                    price = row['price']
                    area = row['area']
                    kind = row['type']
                    law = row['law']
                    bedroom = row['bedroom']
                    floor = row['floor']
                    description = row['description']
                    width = row['width']
                    direction = row['direction']
                    toilet = row['toilet']
                    location = row['address']
                    geocode = row['geo_code']
                    
                    # Handle 'price' column
                    if price.strip().lower() == 'thỏa thuận':
                        price = 0  # or set to a default value, like 0
                    elif not is_decimal(price):
                        price = 0  # or set to a default value

                    # Handle 'width' column
                    if not is_decimal(width):
                        width = 0  # or set to a default value

                    # Handle 'bedroom' column
                    if not bedroom.strip() or bedroom.lower() == 'null':
                        bedroom = 0  # or set to a default value
                       
                    # Handle 'floor' column 
                    if not floor.strip() or floor.lower() == 'null':
                        floor = 0  # or set to a default value

                    # Handle 'toilet' column (if needed)
                    if not is_decimal(toilet):
                        toilet = 0  # or set to a default value

                    # Kiểm tra geocode nếu là None hoặc rỗng, bỏ qua dòng này
                    if not geocode:
                        # print(f"Không có giá trị geocode cho dòng: {row}")
                        continue

                    # Chuyển geo_code thành string nếu nó là list
                    if isinstance(geocode, list):
                        geocode = str(geocode)

                    # Thêm dữ liệu vào bảng pools_bds
                    cursor.execute("""
                        INSERT INTO pools_bds (title, price, area, kind, law, bedroom, floor, address_id, Type_id, description, width, direction, toilet, location, geocode)
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                    """, (title, price, area, kind, law, bedroom, floor, address_id, type_id, description, width, direction, toilet, location, geocode))
                    connection.commit()
                    
                    # Lấy BDS_id của BDS vừa thêm
                    cursor.execute("SELECT LAST_INSERT_ID()")
                    bds_id = cursor.fetchone()['LAST_INSERT_ID()']
                    
                    # Thêm dữ liệu vào bảng pools_image (nếu có)
                    if 'image_links' in row and row['image_links']:  # Kiểm tra xem cột 'image_link' có tồn tại và có giá trị không
                        image_link = row['image_links']
                        cursor.execute("INSERT INTO pools_image (BDS_id, content) VALUES (%s, %s)", (bds_id, image_link))
                        connection.commit()
                        
                except Exception as e:
                    print(f"Lỗi khi xử lý dòng: {row}")
                    print(e)
finally:
    connection.close()
